<!--Document description-->

*This document details the parameterization of COBALT for the Kaneohe (hioekg) grid.*

*This is a live document that will be updated as needed.*

*Last update:* 2019-02-28

# Parameterizing COBALT

## Rivers

The rivers called by chunk #3 in ``setup_inner_frc.py`` include:
  1. Heeia (16275000)
  2. Waiahole (16294100)
  3. Waihee (16284200)
  4. Waikane (16294900)
  
I've structured my literature review and other research about important river constituents to prioritize findings pertaining to any of these specific rivers. If no information exists for a particular constituent from one of these rivers, I have expanded my search to include greater Oahu, followed by consideration of fluvial research across the Main Hawaiian Islands. I have avoided parameterizing the model using broad (i.e., global-scale) generalizations whenever possible.

``setup_inner_frc.py`` produces the river NetCDF and contains the following relevant chunk:

```python 
default_vals = {
    'river_temp': 25,
    'river_salt': 5,
    'river_alk': 0.00141,
    'river_dic': 0.0014,
    'river_ldon': 0.000000054,
    'river_ldop': 0.00000000195,
    'river_nbact': 0.000000398,
    'river_ndet': 0.000025,
    'river_nh4': 0.00000034,
    'river_nlg': 0.00000119,
    'river_no3': 0.0000048,
    'river_nsm': 0.000000523,
    'river_nsmz': 0.0000283,
    'river_o2': 0.000395,
    'river_pdet': 0.00000000161,
    'river_po4': 0.0000007,
    'river_sio4': 0.000157,
    'river_sldon': 0.00000108,
    'river_sldop': 0.000000039,
    'river_srdon': 0.00000427,
    'river_srdop': 0.000000154
}

# ...

    # Create the COBALT variables we want
    seapy.roms.cobalt.add_psource_var(
        nc, ['alk', 'dic', 'ldon', 'ldop', 'nbact', 'ndet', 'nh4', 'nlg', 'no3', 'nsm', 'nsmz', 'o2', 'pdet', 'po4', 'sio4', 'sldon', 'sldop', 'srdon', 'srdop'])```
        
*Note*: according to ``river-hioekg.nc``, units for all are mol/kg.

I am editing ``setup_inner_frc.py`` as I go. I am also editing ``bio_cobalt.in`` as described below.

### bio_cobalt.in

Searched ``LtracerSrc`` to locate the portion of the file that lists the 33 parameters of possible interest. **Reminder** to turn on the switches to write the parameters of interest to the history file, and ensure the model is also considering the tracers (the 2 rows of Ts and Fs should be identical).

Parameters are grouped- somewhat arbitrarily- for readability.

### Plankton (6 parameters)

I have not been able to find fluvial estimates for plankton- which makes sense, as a lot of these guys are marine dwellers. My notes here reflect the **stream mouth** values that the model should reflect, sourced from [Cox et al. 2006](https://www.int-res.com/articles/meps2006/324/m324p019.pdf) (primarily Fig. 2 and Table 3) and sampled at the 'MP' site near the mouth of Kaneohe stream. The authors found difference in "dry" (June-September) and "wet" (October-May) seasons. I will parameterize the model to start Jan. 1 (so, wet season values- dry recorded here for validation).

##### Phytoplankton

Per Stock et al. 2014: All phytoplankton groups take up ammonia and nitrate; diazotrophs take up N2. 

Pico- and nano- eukaryotes: [Ribeiro et al. 2016](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5111892/) note that pico- (0.2- 2 micrometers) and nano- (2 - 20 micrometers) eukaryotes are autotrophic. The mean diameter of nanoeukaryotes was 3.3 micrometers per [Cox et al. 2006](https://www.int-res.com/articles/meps2006/324/m324p019.pdf), so they are grouped here as ``nsm``.

For all calculations, density assumed equivalent to freshwater (working with mid-plume numbers). 

*Prochlorococcus* spp.: 0.5 - 0.7 micrometers, the smallest known photosynthetic organism [Partensky et al. 1999](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC98958/). "The fact that Kāne‘ohe Bay waters were devoid of Prochlorococcus was previously
pointed out in a short-term study by Liu et al. (1995). In our study, Prochlorococcus was only found at the NB site, even though it dominated the oceanic site. The presence of Prochlorococcus at the NB site may reflect advection into the Bay from oceanic waters, even if populations of Prochlorococcus were unable to grow under the environmental conditions found in the northern sector of the Bay."

*Synechococcus* spp.: 0.5 - 0.8 micrometers. [Uysal 2001](https://academic.oup.com/plankt/article/23/2/175/1463509).

  * ``nsm`` Small Phytoplankton Nitrogen: size < 10 micrometers. From [Cox et al. 2006](https://www.int-res.com/articles/meps2006/324/m324p019.pdf), this encompasses: picoeukaryotes, nanoeukaryotes, *Synechococcus* spp. (would include *Prochlorococcus* spp. if it grew in the bay). = **5.23e-7 mol/kg** (Note: dry season is 30% higher for ``nsm``)

In [2]:
### Picoeukaryotes

## Dry season
r = 0.5 * 1 # assumed mean cell radius; value sourced from Ribeiro et al. 2016 
cmL = 41100 # cells per mL, Table 3
cL = cmL * 1000 # convert to cells per L
picoeukaryote_vol = ((4/3) * 3.14 * (r**3)) # calculate volume of picoeukaryote in cubic micrometers
picoeukaryote_fg_N = 26.1 * picoeukaryote_vol # fg N per cell
picoeukaryote_g_N = picoeukaryote_fg_N * 1e-15 # convert fg to g
picoeukaryote_gL = cL * picoeukaryote_g_N # calculate g per L
pico_dry_mol = picoeukaryote_gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
print("Pico-dry:" ,pico_dry_mol)

## Wet season
r = 0.5 * 1 # assumed mean cell radius; value sourced from Ribeiro et al. 2016 
cmL = 27300 # cells per mL, Table 3
cL = cmL * 1000 # convert L
picoeukaryote_vol = ((4/3) * 3.14 * (r**3)) # calculate volume of picoeukaryote in cubic micrometers
picoeukaryote_fg_N = 26.1 * picoeukaryote_vol # fg N per cell
picoeukaryote_g_N = picoeukaryote_fg_N * 1e-15 # convert fg to g
picoeukaryote_gL = cL * picoeukaryote_g_N # calculate g per L
pico_wet_mol = picoeukaryote_gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
print("Pico-wet:" ,pico_wet_mol)

### Nanoeukaryotes

## Dry season
r = 0.5 * 3.3 # mean cell radius; value sourced from Cox et al. 2006 
cmL = 640 # cells per mL, Table 3
cL = cmL * 1000 # convert to cells per L
nanoeukaryote_vol = ((4/3) * 3.14 * (r**3)) # calculate volume of nanoeukaryote in cubic micrometers
nanoeukaryote_fg_N = 26.1 * nanoeukaryote_vol # fg N per volume (= per cell)
nanoeukaryote_g_N = nanoeukaryote_fg_N * 1e-15 # convert fg to g N 
nanoeukaryote_gL = cL * nanoeukaryote_g_N # calculate g per L
nano_dry_mol = nanoeukaryote_gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
print("Nano-dry:" ,nano_dry_mol)

## Wet season
r = 0.5 * 3.3 # mean cell radius; value sourced from Cox et al. 2006 
cmL = 603 # cells per mL, Table 3
cL = cmL * 1000 # convert to cells per L
nanoeukaryote_vol = ((4/3) * 3.14 * (r**3)) # calculate volume of nanoeukaryote in cubic micrometers
nanoeukaryote_fg_N = 26.1 * nanoeukaryote_vol # fg N per volume (= per cell)
nanoeukaryote_g_N = nanoeukaryote_fg_N * 1e-15 # convert fg to g N 
nanoeukaryote_gL = cL * nanoeukaryote_g_N # calculate g per L
nano_wet_mol = nanoeukaryote_gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
print("Nano-wet:" ,nano_wet_mol)

### Synechococcus

## Dry season
cmL = 453000 # cells per mL
cL = cmL * 1000 # convert to L
fgL = cL * 21 # multiply by femtograms of N per cell; value from Table 1, Cox et al. 2006
gL = fgL * 1e-15 # convert fg to g
syn_dry_mol = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
print("Syn-dry:" ,syn_dry_mol)

## Wet season
cmL = 317000 # cells per mL
cL = cmL * 1000 # convert to L
fgL = cL * 21 # multiply by femtograms of N per cell; value from Table 1, Cox et al. 2006
gL = fgL * 1e-15 # convert fg to g
syn_wet_mol = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
print("Syn-wet:" ,syn_wet_mol)

# Sums
wet_sum = pico_wet_mol + nano_wet_mol + syn_wet_mol
print("Wet season nsm:" ,wet_sum)
dry_sum = pico_dry_mol + nano_dry_mol + syn_dry_mol
print("Dry season nsm:" ,dry_sum)

Pico-dry: 4.007974040994668e-08
Pico-wet: 2.6622309323395235e-08
Nano-dry: 2.242873975454604e-08
Nano-wet: 2.113207823748635e-08
Syn-dry: 6.79174966266144e-07
Syn-wet: 4.75272548137677e-07
Wet season nsm: 5.230269356985587e-07
Dry season nsm: 7.416834464306368e-07


Diatoms and *Chaetoceros*: 2 - 500 micrometers. [Ringuet & Mackenzie 2005](https://www.researchgate.net/profile/F_Mackenzie/publication/227303537_Controls_on_Nutrient_and_Phytoplankton_Dynamics_during_Normal_Flow_and_Storm_Runoff_Conditions_Southern_Kaneohe_Bay_Hawaii/links/00b7d5293aa8ee5d17000000.pdf) note that "the phytoplankton community is primarily composed of diatoms, cyanobacteria, and chlorophytes" and that storm pulses caused diatom concentrations to increase 10-fold. "Planktonic cell abundance was in general lower during the wet season, with the exception of opportunistic diatoms that were more abundant during the wet season" [Cox et al. 2006](https://www.int-res.com/articles/meps2006/324/m324p019.pdf). 

Dinoflagellates: 15 - 40 micrometers [https://scripps.ucsd.edu/labs/mlatz/bioluminescence/dinoflagellates-and-red-tides/].
  
  * ``nlg`` Large Phytoplankton Nitrogen: size > 10 micrometers. From [Cox et al. 2006](https://www.int-res.com/articles/meps2006/324/m324p019.pdf), this encompasses: diatoms and *Chaetoceros*, dinoflagellates. = **1.19e-6 mol/kg** (Note: dry season is 25% higher for ``nlg``)

  * ``ndi`` Diazotroph (bacteria/archaea that fix atmospheric N2 into a more usable form such as ammonia) Nitrogen: **N/A fluvial**. See bay-wide section.

In [19]:
### Diatoms

## Dry season
r = 0.5 * 75 # cell radius derived from Table 2, Cox et al. 2006
cL = 4320 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_diatoms_dry = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
print("Diatoms-dry:" ,mol_diatoms_dry)

## Wet season
r = 0.5 * 75 # cell radius derived from Table 2, Cox et al. 2006
cL = 3260 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_diatoms_wet = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
print("Diatoms-wet:" ,mol_diatoms_wet)

### Chaetoceros

## Dry season
r = 0.5 * 15 # cell radius derived from Table 2, Cox et al. 2006
cL = 760 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_ch_dry = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
print("Ch-dry:" ,mol_ch_dry)

## Wet season
r = 0.5 * 15 # cell radius derived from Table 2, Cox et al. 2006
cL = 1370 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_ch_wet = gL/14.0067 # convert to mol/kg by dividing by molecular weight of 
print("Ch-wet:" ,mol_ch_wet)

### Dinoflagellates

## Dry season
r = 0.5 * 22 # cell radius in micrometers derived from Table 2, Cox et al. 2006
cL = 3250 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_dino_dry = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
print("Dino-dry:" ,mol_dino_dry)

## Wet season
r = 0.5 * 22 # cell radius derived from Table 2, Cox et al. 2006
cL = 1630 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_dino_wet = gL/14.0067 # convert to mol/kg by dividing by molecular weight of 
print("Dino-wet:" ,mol_dino_wet)

# Sums
wet_sum = mol_diatoms_wet + mol_ch_wet + mol_dino_wet
print("Wet season nlg:" ,wet_sum)
dry_sum = mol_diatoms_dry + mol_ch_dry + mol_dino_dry
print("Dry season nlg:" ,dry_sum)

Diatoms-dry: 1.5543531047795778e-06
Diatoms-wet: 1.1729609077734779e-06
Ch-dry: 1.95767496558348e-09
Ch-wet: 3.5289667142754833e-09
Dino-dry: 2.7119488201514872e-08
Dino-wet: 1.3601466390298227e-08
Wet season nlg: 1.1900913408780515e-06
Dry season nlg: 1.5834302679466762e-06


##### Zooplankton

We tuned our model to more precisely reflect the planktonic community characteristics of Kaneohe Bay ([Cox et al. 2006](https://www.int-res.com/articles/meps2006/324/m324p019.pdf)).
[Cox et al. 2006](https://www.int-res.com/articles/meps2006/324/m324p019.pdf) parse their zooplankton concentrations by size: large (> 183 micrometers), medium (63 - 183 micrometers), and small (< 63 micrometers). We combined the small and medium zooplankton measurements of Cox et al. (2006) and classified the associated N contribution for small zooplankton (per COBALT specifications, < 200 micrometers). 
  
Ciliates: mean 25 micrometers. [Cox et al. 2006](https://www.int-res.com/articles/meps2006/324/m324p019.pdf). Because ciliates are *primarily* heterotrophic, I am including them in the estimation of nsmz. cells/ml = 554 (dry season) or 562 (wet season). 
*Dry season*: 554/totalCells (total cells for ``nsm``) = percent% of the total N = percentAsDecimal x nsm N (micrograms/L) = estimate (micrograms/L). Estimate/1000 = newEst (g/L) x (mol/14.0067 g)(molecular weight of N) = newEst2 mol/kg
*Wet season*: 562/totalCells (total cells for ``nsm``) = percent% of the total N = percentAsDecimal x nsm N (micrograms/L) = estimate (micrograms/L). Estimate/1000 = newEst (g/L) x (mol/14.0067 g)(molecular weight of N) = newEst2 mol/kg 


  * ``nsmz`` Small Zooplankton Nitrogen: size < 200 micrometers; "microzooplankton". Encompasses ciliates. = **2.83e-5**. No zooplankton data for "MP" (stream mouth) location.
 
  * ``nmdz`` Medium-sized Zooplankton Nitrogen: size 200 - 2000 micrometers; "mesozooplankton" = small/med copepods.
  
  * ``nlgz`` Large Zooplankton Nitrogen: 2 - 20 mm; large copepods and krill.

In [3]:
### Ciliates

## Dry season
cL = 554 # cells per L, Table 3
ugC = 4.67 # micrograms C per ciliate cell in Northern Gulf of Mexico; Table 3, Strom & Strom 1996
ugN = ugC * (16/106) # use of the Redfield ratio to convert to micrograms N per cell
gN = ugN/1e6 # convert to g N per cell
gL = cL * gN # g per L
mol_cil_dry = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
print("Cil-dry:" ,mol_cil_dry)

## Wet season
cL = 562 # cells per L, Table 3
ugC = 4.67 # micrograms C per ciliate cell in Northern Gulf of Mexico; Table 3, Strom & Strom 1996
ugN = ugC * (16/106) # use of the Redfield ratio to convert to micrograms N per cell
gN = ugN/1e6 # convert to g N per cell
gL = cL * gN # g per L
mol_cil_wet = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
print("Cil-wet:" ,mol_cil_wet)


Cil-dry: 2.7880781044004412e-05
Cil-wet: 2.8283391600596534e-05


#### Semi-refractory, semi-labile, labile N and P (6 parameters)

  * ``ldon`` labile DON: 0.01 x (5.4 x 10-6 mol/kg) = **5.40e-8 mol/kg**
  * ``sldon`` Semilabile DON: 0.20 x (5.4 x 10-6 mol/kg) = **1.08e-6 mol/kg**
  * ``srdon`` Semi-Refractory DON: 0.79 x (5.4 x 10-6 mol/kg) = **4.27e-6 mol/kg**
  
  * ``ldop`` labile DOP: 0.01 x (1.95 x 10-7 mol/kg) = **1.95e-9 mol/kg**
  * ``sldop`` Semilabile DOP: 0.20 x (1.95 x 10-7 mol/kg) = **3.90e-8 mol/kg**
  * ``srdop`` Semi-Refractory DOP: 0.79 x (1.95 x 10-7 mol/kg) = **1.54e-7 mol/kg**
  
    * Per [Hoover & Mackenzie 2009](https://www.researchgate.net/publication/225712771_Fluvial_Fluxes_of_Water_Suspended_Particulate_Matter_and_Nutrients_and_Potential_Impacts_on_Tropical_Coastal_Water_Biogeochemistry_Oahu_Hawai'i): DON = 5.4 micromol/L = **5.40e-6 mol/kg** and DOP = 0.195 micromol/L = **1.95e−7 mol/kg**. 
    * These measurements support [Ringuet & Mackenzie 2005](https://www.researchgate.net/profile/F_Mackenzie/publication/227303537_Controls_on_Nutrient_and_Phytoplankton_Dynamics_during_Normal_Flow_and_Storm_Runoff_Conditions_Southern_Kaneohe_Bay_Hawaii/links/00b7d5293aa8ee5d17000000.pdf): DON = 7.44 (average) micromol/L = **7.44e-6 mol/kg** and DOP = 0.3 (average) micromol/L = **3.00e-7 mol/kg**.

  * Per [Wilson & Xenopoulous 2008](https://www.researchgate.net/profile/Marguerite_Xenopoulos/publication/248828330_Effects_of_agricultural_land_use_on_the_composition_of_fluvial_dissolved_organic_matter/links/00b4952121ab928d14000000.pdf): "...higher ambient nitrogen concentrations might be expected to disproportionately increase respiration. This increased availability could result in terrestrial DOM moving shorter distances before either being respired or incorporated within stream organisms or sediments. At broader scales, this trend could result in a reduction in the overall amount of DOM reaching downstream systems." Study area = Ontario.

  * [Mann et al. 2012](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2011JG001798): "DON concentrations were highly correlated to DOC and thus followed the same trend during the study period (r = 0.92; p < 0.001; n = 18). DON ranged from ∼0.1 mg L−1 under the ice (mean = 0.09 mg L−1, n = 3) during preflush conditions to 0.6 mg L−1 (mean = 0.37 mg L−1, n = 14) during the freshet." Study area = Siberia.

  * [Hansell et al. 2009](https://darchive.mblwhoilibrary.org/bitstream/handle/1912/3183/22-4_hansell.pdf?sequence=1): extensive paper on marine DOC. From the paper: "This biologically labile fraction represents a large flux of carbon in the ocean, but with rapid turnover it constitutes a very small fraction (< 1%) of the ocean DOC inventory. A more biologically resistant fraction, produced in the euphotic zone at 15–20% of net community productivity (~ 2 Pg C yr-1; Hansell and Carlson, 1998b), is not immediately mineralized and instead accumulates in the surface ocean as biologically semi-labile DOC (Carlson, 2002; Hansell, 2002)."
  
  * Per [Stock et al. 2014](https://www.sciencedirect.com/science/article/pii/S0079661113001079): "Nitrogen is the biomass variable in COBALT and is assumed to be combined with carbon according to the Redfield ratio (106:16, Redfield et al., 1963) for all organic material." 
  
Given the inherent adherence of COBALT to the Redfield ratio (106:16; Redfield et al. 1963), we assumed that the lability of DON would correlate closely with measurements of C (e.g., Mann et al. 2012). Per Hansell et al. (2009), we assumed that 1% of fluvial DON could be classified as labile (i.e., biologically available); this estimate supports the assertion that labile DOM is subject to respiration or incorporation into riverine organisms or sediments (Wilson & Xenopoulous 2008). We assumed that 20% of DON could be classified as semi-labile (i.e., seasonal rates of decay) with the remaining 79% of all DOM classified as semi-refractory (i.e., decadal decay rates) (Hansell et al. 2009). We applied these assumptions to our determination of fluvial DOP as well.

#### More nitrogen budget (4 parameters)

  * ``nbact`` Explicit free-living heterotrophic bacteria compartment:  [Stock et al. 2014](https://www.sciencedirect.com/science/article/pii/S0079661113001079): "The free-living bacteria in COBALT 1.0 function only as remineralizers of dissolved organic nutrients and don’t compete with phytoplankton for inorganic nutrients (e.g., Antia et al., 1991)." = **3.98e-7 mol/kg**[Cox et al. 2006](https://www.int-res.com/articles/meps2006/324/m324p019.pdf)
    
  * ``nh4`` Ammonia: 0.34 micromol N/L = **3.40e-7 mol/kg** [Hoover & Mackenzie 2009](https://www.researchgate.net/publication/225712771_Fluvial_Fluxes_of_Water_Suspended_Particulate_Matter_and_Nutrients_and_Potential_Impacts_on_Tropical_Coastal_Water_Biogeochemistry_Oahu_Hawai'i). This value was measured directly from He'eia stream.
     
  * ``no3`` Nitrate: 4.8 micromol N/L = **4.80e-6 mol/kg** [Hoover & Mackenzie 2009](https://www.researchgate.net/publication/225712771_Fluvial_Fluxes_of_Water_Suspended_Particulate_Matter_and_Nutrients_and_Potential_Impacts_on_Tropical_Coastal_Water_Biogeochemistry_Oahu_Hawai'i). This value was measured directly from He'eia stream.

  * ``ndet`` Nitrogen detritus produced by phytoplankton aggregation and a fraction of ingestion from zooplankton and higher predators. 25 micromol N/L = **2.50e-5 mol/kg** Per [Tanaka & Mackenzie 2005](https://www.researchgate.net/profile/F_Mackenzie/publication/265088005_Ecosystem_behavior_of_southern_Kaneohe_Bay_A_statistical_and_modeling_approach/links/566f30a708aec0bb67bf0739.pdf), "the concentration of particulate N in the stream is 25 micromol N/L, which is assumed to be equivalent to the concentration of particulate organic nitrogen." 

In [28]:
### Heterotrophic bacteria (nbact)

## Dry season
cmL = 3800000 # cells per mL; value from Table 3, Cox et al. 2006
cL = cmL * 1000 # convert to L
fgL = cL * 1.8 # multiply by femtograms of N per cell; value from Table 1, Cox et al. 2006
gL = fgL * 1e-15 # convert fg to g
mol = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
print("nbact-dry:" ,mol)

## Wet season
cmL = 3100000 # cells per mL; value from Table 3, Cox et al. 2006
cL = cmL * 1000 # convert to L
fgL = cL * 1.8 # multiply by femtograms of N per cell; value from Table 1, Cox et al. 2006
gL = fgL * 1e-15 # convert fg to g
mol = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
print("nbact-wet:" ,mol)

nbact-dry: 4.883377240891859e-07
nbact-wet: 3.9838077491486223e-07


#### Iron (5 parameters)

  * ``fesm`` Small Phytoplankton Iron
  * ``fedi`` Diazotroph Iron
  * ``felg`` Large Phytoplankton Iron
  * ``fed`` Dissolved Iron
  * ``fedet`` Detrital Iron

No notes yet.

#### Sand/detritus (7 parameters)

  * ``sio4`` Silicate: 1.4 x 10-5 mol/kg to 1.73 x 10-4 mol/kg [Ringuet & Mackenzie 2005](https://www.researchgate.net/profile/F_Mackenzie/publication/227303537_Controls_on_Nutrient_and_Phytoplankton_Dynamics_during_Normal_Flow_and_Storm_Runoff_Conditions_Southern_Kaneohe_Bay_Hawaii/links/00b7d5293aa8ee5d17000000.pdf); source = multiple buoys across Kaneohe Bay. Starting parameterization with higher (mean = 1.57 x 10-4 mol/kg) number.
  * ``silg`` Large Phytoplankton Silicon
  * ``sidet`` Detrital Silicon
  * ``cadet_arag`` Detrital CaCO3: Per Stock et al. 2014, "Aragonite detritus (cadet_arag) is assumed to dissolve immediately upon reaching the sediment." Ignoring for now...
  * ``cadet_calc`` Detrital CaCO3: Per Stock et al. 2014, "Calcite, however, is more resistant to dissolution and long-term balances of calcite supply to the sediments, dissolution to overlying waters, and burial has the potential to influence ocean chemistry. The calcite concentration in the sediments is thus tracked in COBALT using the variable cased (moles CaCO 3 m -3 ) and burial and dissolution rates are based on the metamodel of Dunne et al. (2012)." 
  
As of 2019-02-08: K. Fagan is now KS; corresponding with A. K. (student) regarding her soon-to-be-published work on this. She has measurements of CaCO3 in mol and now determining if they are fluvial, bay-wide (reference for model output), and if she can distinguish between aragonite and calcite.
  
  * ``lith`` Lithogenic Aluminosilicate
  * ``lithdet`` lithdet



#### Etc.

  * ``po4`` Phosphate: "...measured as soluble reactive phosphorus (SRP) but often assumed equivalent to PO4^3- (phosphate)"; 0.7 micromol = **7.00e-7 mol/kg** (Table 2) [Hoover & Mackenzie 2009](https://www.researchgate.net/publication/225712771_Fluvial_Fluxes_of_Water_Suspended_Particulate_Matter_and_Nutrients_and_Potential_Impacts_on_Tropical_Coastal_Water_Biogeochemistry_Oahu_Hawai'i); 0.13 to 0.64 micromol at stream mouth = **1.33e-7 to 6.56e-7 mol/kg** [Ringuet & Mackenzie 2005](https://www.researchgate.net/profile/F_Mackenzie/publication/227303537_Controls_on_Nutrient_and_Phytoplankton_Dynamics_during_Normal_Flow_and_Storm_Runoff_Conditions_Southern_Kaneohe_Bay_Hawaii/links/00b7d5293aa8ee5d17000000.pdf); source = multiple buoys across Kaneohe Bay.


  * ``pdet`` Detrital Phosphorus: "Particulate organic phosphorus (POP) calculated by difference from total and inorganic constituents": 0.23% of SRP (Table 3); 0.7 micromol SRP (Table 2) x 0.0023 = 0.00161 micromol POP = **1.61e-9 mol/kg**  [Hoover & Mackenzie 2009](https://www.researchgate.net/publication/225712771_Fluvial_Fluxes_of_Water_Suspended_Particulate_Matter_and_Nutrients_and_Potential_Impacts_on_Tropical_Coastal_Water_Biogeochemistry_Oahu_Hawai'i)
  
  
  * ``o2`` Oxygen: ~110% saturation at Kaneohe Stream mouth at 25 Celsius [converts](http://www.waterontheweb.org/under/waterquality/DOSatCalc.html) to 9.09 mg/L = 9.09e-3 g/L = **3.95e-4 mol/kg** [Ringuet 2003](https://scholarspace.manoa.hawaii.edu/bitstream/10125/7044/2/uhm_ms_3841_r.pdf)
 
 
  * ``dic`` Dissolved Inorganic Carbon:  1400 micromol/kg (average) = **1.40e-3 mol/kg** (Kaneohe stream; at stream mouth, 1700 micromol/kg = 1.70e-3 mol/kg  [Fagan & Mackenzie 2007](https://www.researchgate.net/profile/F_Mackenzie/publication/253669603_Processes_Controlling_Air-Sea_Exchange_of_CO2_in_a_Subtropical_Pacific_Estuary/links/58dc716ca6fdcc7c9f3d738d/Processes-Controlling-Air-Sea-Exchange-of-CO2-in-a-Subtropical-Pacific-Estuary.pdf); sampling location = Kaneohe Bay surface waters and stream
    * = 0.443 mmol/kg = 4.43e-4 (average) mol/kg [Paquay et al. 2007](https://orbi.uliege.be/bitstream/2268/2597/1/paquay_et_al_2007_in_press.pdf); sampling location = rivers feeding into Hilo Bay. 
  
  
  * ``alk`` Alkalinity: 1410 micromol/kg (average) = **1.41 x 10-3 mol/kg** (Kaneohe stream); at stream mouth, 1950 micromol/kg = 1.95 x 10-3 mol/kg [Fagan & Mackenzie 2007](https://www.researchgate.net/profile/F_Mackenzie/publication/253669603_Processes_Controlling_Air-Sea_Exchange_of_CO2_in_a_Subtropical_Pacific_Estuary/links/58dc716ca6fdcc7c9f3d738d/Processes-Controlling-Air-Sea-Exchange-of-CO2-in-a-Subtropical-Pacific-Estuary.pdf); sampling location = Kaneohe Bay surface waters and stream. 
    * 0.376 mmol/kg = 3.76e-4 (average) mol/kg [Paquay et al. 2007](https://orbi.uliege.be/bitstream/2268/2597/1/paquay_et_al_2007_in_press.pdf); sampling location = rivers feeding into Hilo Bay. 


##### Baywide reference values

   * 1 nsm Small Phytoplankton Nitrogen (picoeukaryotes, nanoeukaryotes, Synechococcus spp.): 
        * **timeseries**
       * Bay-wide: 0.04 + 0.01 + 0.92 micromol N/L = 9.7e-7 * 1.024 (ocean water density conversion) = **9.93e-7 mol/kg** [Ribes et al. 2003](https://www.int-res.com/articles/meps2003/257/m257p013.pdf)
   * 2 nlg Large Phytoplankton Nitrogen (diatoms and Chaetoceros, dinoflagellates):
       * **timeseries**
       * Bay-wide: 0.04 + 0.02 micromol N/L = 6.00e-8 * 1.024 = **6.14e-8* mol/kg** [Ribes et al. 2003](https://www.int-res.com/articles/meps2003/257/m257p013.pdf)
   * 3 ndi Diazotroph Nitrogen: Per [Landry et al. 2984](), "Using a volume-carbon conversion factor (e.g. Strickland, 1960), we estimate that total community carbon was about 86 pg C/L' of which Chlorella sp. accounted for 47%, cyanobacteria 11%, heterotrophic bacteria 9% and combined flagellate populations 23%." From Table 1: per cell C content = 0.1 pg = (Redfield ratio) = 0.01509 pg of N per cell. 104,000 cells (mean) per L = 1569.36 pg/L = 1.57e-9 g/L /14.0067 (molecular weight of N) = **1.12e-10 mol/kg**
   * 4 nsmz Small Zooplankton Nitrogen (ciliates): **timeseries**
   * 5 nmdz Medium zooplankton Nitrogen: **timeseries**
   * 6 nlgz Large Zooplankton Nitrogen: **timeseries**
   * 7 ldon labile DON: Bay-wide (average): 7.84 micromol = 0.01 x (8.04e-6 mol/kg) = **8.04e-8 mol/kg**  [Ringuet & Mackenzie 2005](https://www.researchgate.net/profile/F_Mackenzie/publication/227303537_Controls_on_Nutrient_and_Phytoplankton_Dynamics_during_Normal_Flow_and_Storm_Runoff_Conditions_Southern_Kaneohe_Bay_Hawaii/links/00b7d5293aa8ee5d17000000.pdf); source = multiple buoys across Kaneohe Bay. These are values that reflect the marine surface concentration.
   * 8 sldon Semilabile DON: 7.84 micromol = 0.20 x (8.04 x 10-6 mol/kg) = **1.61e-6 mol/kg**  [Ringuet & Mackenzie 2005](https://www.researchgate.net/profile/F_Mackenzie/publication/227303537_Controls_on_Nutrient_and_Phytoplankton_Dynamics_during_Normal_Flow_and_Storm_Runoff_Conditions_Southern_Kaneohe_Bay_Hawaii/links/00b7d5293aa8ee5d17000000.pdf); source = multiple buoys across Kaneohe Bay. These are values that reflect the marine surface concentration.
   * 9 srdon Semi-Refractory DON: 7.84 micromol = 0.79 x (8.04 x 10-6 mol/kg) = **6.35e-6 mol/kg**  [Ringuet & Mackenzie 2005](https://www.researchgate.net/profile/F_Mackenzie/publication/227303537_Controls_on_Nutrient_and_Phytoplankton_Dynamics_during_Normal_Flow_and_Storm_Runoff_Conditions_Southern_Kaneohe_Bay_Hawaii/links/00b7d5293aa8ee5d17000000.pdf); source = multiple buoys across Kaneohe Bay. These are values that reflect the marine surface concentration.
   * 10 nbact bacterial: 
      * **timeseries**
      * Bay-wide (SB): 0.21 micromol N/L = **2.15e-7 mol/kg** [Tanaka & Mackenzie 2005](https://www.researchgate.net/profile/F_Mackenzie/publication/265088005_Ecosystem_behavior_of_southern_Kaneohe_Bay_A_statistical_and_modeling_approach/links/566f30a708aec0bb67bf0739.pdf)
      * Bay-wide (CB): 0.34 micromol N/L = **3.49e-7 mol/kg** [Tanaka & Mackenzie 2005](https://www.researchgate.net/profile/F_Mackenzie/publication/265088005_Ecosystem_behavior_of_southern_Kaneohe_Bay_A_statistical_and_modeling_approach/links/566f30a708aec0bb67bf0739.pdf)
      * Bay-wide: 0.61 micromol N/L = **6.10e-7 mol/kg** [Ribes et al. 2003](https://www.int-res.com/articles/meps2003/257/m257p013.pdf)
   * 11 nh4 Ammonia: 
      * **timeseries**
      * Bay-wide (average): .218 micromol N/L = **2.23e-7 mol/kg** [Ringuet & Mackenzie 2005](https://www.researchgate.net/profile/F_Mackenzie/publication/227303537_Controls_on_Nutrient_and_Phytoplankton_Dynamics_during_Normal_Flow_and_Storm_Runoff_Conditions_Southern_Kaneohe_Bay_Hawaii/links/00b7d5293aa8ee5d17000000.pdf); source = multiple buoys across Kaneohe Bay. These are values that reflect the marine surface concentration.
   * 12 no3 Nitrate: Bay-wide (average): 
     * **timeseries**
     * 0.204 micromol N/L = **2.091e-7 mol/kg** [Ringuet & Mackenzie 2005](https://www.researchgate.net/profile/F_Mackenzie/publication/227303537_Controls_on_Nutrient_and_Phytoplankton_Dynamics_during_Normal_Flow_and_Storm_Runoff_Conditions_Southern_Kaneohe_Bay_Hawaii/links/00b7d5293aa8ee5d17000000.pdf); source = multiple buoys across Kaneohe Bay. These are values that reflect the marine surface concentration.
   * 13 ndet ndet: from Table 1, using values representing pelagic autotrophs.
     * Bay-wide (southern basin): 0.67 micromol N/L = **6.87e-7 mol/kg** [Tanaka & Mackenzie 2005](https://www.researchgate.net/profile/F_Mackenzie/publication/265088005_Ecosystem_behavior_of_southern_Kaneohe_Bay_A_statistical_and_modeling_approach/links/566f30a708aec0bb67bf0739.pdf)
     *  Bay-wide (central basin): 0.32 micromol N/L = **3.28 x 10-7  mol/kg** [Tanaka & Mackenzie 2005](https://www.researchgate.net/profile/F_Mackenzie/publication/265088005_Ecosystem_behavior_of_southern_Kaneohe_Bay_A_statistical_and_modeling_approach/links/566f30a708aec0bb67bf0739.pdf)
   * 14 sio4 Silicate: Bay-wide (average): 
     * **timeseries**
     * 7.28 micromol = **7.46e-6 mol/kg** [Ringuet & Mackenzie 2005](https://www.researchgate.net/profile/F_Mackenzie/publication/227303537_Controls_on_Nutrient_and_Phytoplankton_Dynamics_during_Normal_Flow_and_Storm_Runoff_Conditions_Southern_Kaneohe_Bay_Hawaii/links/00b7d5293aa8ee5d17000000.pdf); source = multiple buoys across Kaneohe Bay.
   * 15 silg Large Phytoplankton Silicon
   * 16 sidet Detrital Silicon
   * 17 cadet_calc Detrital CaCO3: reached out to A. A. at Scripps about this.
   * 18 cadet_arag Detrital CaCO3: reached out to A. A. at Scripps about this.
   * 19 lith Lithogenic Aluminosilicate
   * 20 lithdet lithdet
   * 21 ldop labile DOP: Bay-wide (average): 0.148 micromol = 0.01 x (1.52 x 10-7 mol/kg) = **1.52e-9 mol/kg**  [Ringuet & Mackenzie 2005](https://www.researchgate.net/profile/F_Mackenzie/publication/227303537_Controls_on_Nutrient_and_Phytoplankton_Dynamics_during_Normal_Flow_and_Storm_Runoff_Conditions_Southern_Kaneohe_Bay_Hawaii/links/00b7d5293aa8ee5d17000000.pdf); source = multiple buoys across Kaneohe Bay. These are values that reflect the marine surface concentration.
   * 22 sldop Semilabile DOP: Bay-wide (average): 0.148 micromol = 0.20 x (1.52 x 10-7 mol/kg) = **3.04e-8 mol/kg**  [Ringuet & Mackenzie 2005](https://www.researchgate.net/profile/F_Mackenzie/publication/227303537_Controls_on_Nutrient_and_Phytoplankton_Dynamics_during_Normal_Flow_and_Storm_Runoff_Conditions_Southern_Kaneohe_Bay_Hawaii/links/00b7d5293aa8ee5d17000000.pdf); source = multiple buoys across Kaneohe Bay. These are values that reflect the marine surface concentration.
   * 23 srdop Semi-Refractory DOP: Bay-wide (average): 0.148 micromol = 0.79 x (1.52 x 10-7 mol/kg) = **1.20e-7 mol/kg**  [Ringuet & Mackenzie 2005](https://www.researchgate.net/profile/F_Mackenzie/publication/227303537_Controls_on_Nutrient_and_Phytoplankton_Dynamics_during_Normal_Flow_and_Storm_Runoff_Conditions_Southern_Kaneohe_Bay_Hawaii/links/00b7d5293aa8ee5d17000000.pdf); source = multiple buoys across Kaneohe Bay. These are values that reflect the marine surface concentration.
   * 24 po4 Phosphate: Bay-wide (average): 0.122 micromol = **1.25e-7 mol/kg** [Ringuet & Mackenzie 2005](https://www.researchgate.net/profile/F_Mackenzie/publication/227303537_Controls_on_Nutrient_and_Phytoplankton_Dynamics_during_Normal_Flow_and_Storm_Runoff_Conditions_Southern_Kaneohe_Bay_Hawaii/links/00b7d5293aa8ee5d17000000.pdf); source = multiple buoys across Kaneohe Bay. These are values that reflect the marine surface concentration.
   * 25 pdet Detrital Phosphorus
   * 26 fesm Small Phytoplankton Iron
   * 27 fedi Diazotroph Iron
   * 28 felg Large Phytoplankton Iron
   * 29 fed Dissolved Iron
   * 30 fedet Detrital Iron
   * 31 o2 Oxygen: 250 micromol/kg = **2.50e-4 mol/kg** [Andersson et al. 2009](https://www.biogeosciences.net/6/1811/2009/bg-6-1811-2009.pdf); control = simulation of bay environment.
   * 32 dic Dissolved Inorganic Carbon: Bay-wide (average): 1913 micromol/kg = **1.91e-3 mol/kg** [Fagan & Mackenzie 2007](https://www.researchgate.net/profile/F_Mackenzie/publication/253669603_Processes_Controlling_Air-Sea_Exchange_of_CO2_in_a_Subtropical_Pacific_Estuary/links/58dc716ca6fdcc7c9f3d738d/Processes-Controlling-Air-Sea-Exchange-of-CO2-in-a-Subtropical-Pacific-Estuary.pdf)
   * 33 alk Alkalinity: Bay-wide (average): 2170 micromol/kg = **2.17e-3 mol/kg** [Fagan & Mackenzie 2007](https://www.researchgate.net/profile/F_Mackenzie/publication/253669603_Processes_Controlling_Air-Sea_Exchange_of_CO2_in_a_Subtropical_Pacific_Estuary/links/58dc716ca6fdcc7c9f3d738d/Processes-Controlling-Air-Sea-Exchange-of-CO2-in-a-Subtropical-Pacific-Estuary.pdf)

##### Timeseries notes

  * [Cox et al. 2006](https://www.int-res.com/articles/meps2006/324/m324p019.pdf): "There were no obvious seasonal patterns in nutrient levels throughout the 3 yr CISNet sampling program at any of the 4 main study sites (www.hawaii.edu/cisnet). However, if the CISNet data were simply grouped into dry and wet seasons (dry: June to September; wet: October to May), we did find some significant differences in nutrient concentrations within Kāne‘ohe Bay. Phosphate and TDP were higher during the wet season (Table 3). Significantly higher nitrate concentrations during the wet season were only found at the NB site. In contrast, higher silicate concentrations during the dry season were found at the CB and SB sites (Table 3). The 0.2 μm chl a fraction was significantly higher in the dry season at the NB, CB and SB sites, but all other chl a fractions and the ratio of small to large chl a showed no significant dry versus wet season differences (Table 3). The microbial groups also showed significant differences, with higher concentrations of heterotrophic bacteria, Synechococcus, and picoeukaryotes during the dry season (Table 3). For larger cells, the only significant dry versus wet season differences occurred in dinoflagellate abundances at the SB and MP sites (Table 3b). Zooplankton showed higher biomass (AFDM) during the dry season in the large fraction at SB, and in the medium fraction at CB (Table 3)."
  * [Cox et al. 2006](https://www.int-res.com/articles/meps2006/324/m324p019.pdf) provides wet/dry means for NB, CB, SB, and Kaneohe stream mouth for the following parameters: 
    * PO43: Bay-wide, wet season = 0.08 micromol/L = **8.00e-8 mol/kg** = 39% higher than bay-wide dry season mean (0.049 micromol/L = 4.90e-8 mol/kg)
    * NO3: 
      * NB: wet season = 0.34 micromol/L = **3.40e-7 mol/kg** = 59% higher than dry season (1.40e-7 mol/kg)
      * CB: wet season = 0.10 micromol/L = **1.00e-7 mol/kg** = 30% higher than dry season (7.00e-8 mol/kg)
      * SB: wet season = 0.06 micromol/L = **6.00e-8 mol/kg** = 18% higher than dry season (4.00e-8 mol/kg)
    * NH4: Bay-wide, between 0.09 - 0.15 micromol/L = **9.00e-8 - 1.5e-7 mol/kg**
    * SiO4:
      * NB: wet season = 6.8 micromol/L = **6.80e-6 mol/kg** = 11% lower than dry season (7.60e-6 mol/kg)
      * CB: wet season = 4.5 micromol/L = **4.50e-6 mol/kg** = 25% lower than dry season (6.00e-6 mol/kg)
      * SB: wet season = 6.5 micromol/L = **6.50e-6 mol/kg** = 33% higher than dry season (7.90e-6 mol/kg)
    * All plankton and bacteria:
       * nbact: dry season = 25-30% higher across all portions of the bay.
       * nsm: dry season = 20% higher in CB, SB; 80% higher in NB.
       * nsmz: wet season = 24% higher in NB; 9% higher in CB; 36% higher in SB. NB/SB have very similar values.
       * nlg: dry season = 30% higher in NB, 70% higher in CB, 30% lower in SB
     
    * TDP, TDN, POC, PON

In [18]:
#### Bay-wide timeseries calculations...

### nsmz: Ciliates (NB, CB, SB)
## Dry season
cL = 528 # cells per L, Table 3
ugC = 4.67 # micrograms C per ciliate cell in Northern Gulf of Mexico; Table 3, Strom & Strom 1996
ugN = ugC * (16/106) # use of the Redfield ratio to convert to micrograms N per cell
gN = ugN/1e6 # convert to g N per cell
gL = cL * gN # g per L
mol_cil_dry_NB = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
cL = 685 # cells per L, Table 3
ugC = 4.67 # micrograms C per ciliate cell in Northern Gulf of Mexico; Table 3, Strom & Strom 1996
ugN = ugC * (16/106) # use of the Redfield ratio to convert to micrograms N per cell
gN = ugN/1e6 # convert to g N per cell
gL = cL * gN # g per L
mol_cil_wet_NB = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Dry season
cL = 397 # cells per L, Table 3
ugC = 4.67 # micrograms C per ciliate cell in Northern Gulf of Mexico; Table 3, Strom & Strom 1996
ugN = ugC * (16/106) # use of the Redfield ratio to convert to micrograms N per cell
gN = ugN/1e6 # convert to g N per cell
gL = cL * gN # g per L
mol_cil_dry_CB = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
cL = 440 # cells per L, Table 3
ugC = 4.67 # micrograms C per ciliate cell in Northern Gulf of Mexico; Table 3, Strom & Strom 1996
ugN = ugC * (16/106) # use of the Redfield ratio to convert to micrograms N per cell
gN = ugN/1e6 # convert to g N per cell
gL = cL * gN # g per L
mol_cil_wet_CB = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Dry season
cL = 456 # cells per L, Table 3
ugC = 4.67 # micrograms C per ciliate cell in Northern Gulf of Mexico; Table 3, Strom & Strom 1996
ugN = ugC * (16/106) # use of the Redfield ratio to convert to micrograms N per cell
gN = ugN/1e6 # convert to g N per cell
gL = cL * gN # g per L
mol_cil_dry_SB = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
cL = 723 # cells per L, Table 3
ugC = 4.67 # micrograms C per ciliate cell in Northern Gulf of Mexico; Table 3, Strom & Strom 1996
ugN = ugC * (16/106) # use of the Redfield ratio to convert to micrograms N per cell
gN = ugN/1e6 # convert to g N per cell
gL = cL * gN # g per L
mol_cil_wet_SB = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## nsmz: small & med zoos per Cox et al. 2006 (< 183 micrometers) (NB, CB, SB)
## Dry season
mgm3 = 19.6 + 17.2 # mg of small + med zoos per cubic meter 
gN = 20/1000 # convert mg to g of N per g of zooplankton biomass (NB); Fig. 5
gm3 = mgm3/1000 # convert to g biomass per cubic meter
gNm3 = gm3 * gN # g N contributed by small/med zoos per cubic meter
gL = gNm3 * 1000 # g N per L
mol_smz_dry_NB = gl/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
mgm3 = 14.6 + 16.7 # mg of small + med zoos per cubic meter 
gN = 20/1000 # convert mg to g of N per g of zooplankton biomass (NB); Fig. 5
gm3 = mgm3/1000 # convert to g biomass per cubic meter
gNm3 = gm3 * gN # g N contributed by small/med zoos per cubic meter
gL = gNm3 * 1000 # g N per L
mol_smz_wet_NB = gl/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Dry season
mgm3 = 27 + 11 # mg of small + med zoos per cubic meter 
gN = 20/1000 # convert mg to g of N per g of zooplankton biomass (NB); Fig. 5
gm3 = mgm3/1000 # convert to g biomass per cubic meter
gNm3 = gm3 * gN # g N contributed by small/med zoos per cubic meter
gL = gNm3 * 1000 # g N per L
mol_smz_dry_CB = gl/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
mgm3 = 18.1 + 10.4 # mg of small + med zoos per cubic meter 
gN = 20/1000 # convert mg to g of N per g of zooplankton biomass (NB); Fig. 5
gm3 = mgm3/1000 # convert to g biomass per cubic meter
gNm3 = gm3 * gN # g N contributed by small/med zoos per cubic meter
gL = gNm3 * 1000 # g N per L
mol_smz_wet_CB = gl/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Dry season
mgm3 = 34.8 + 13.2 # mg of small + med zoos per cubic meter 
gN = 20/1000 # convert mg to g of N per g of zooplankton biomass (NB); Fig. 5
gm3 = mgm3/1000 # convert to g biomass per cubic meter
gNm3 = gm3 * gN # g N contributed by small/med zoos per cubic meter
gL = gNm3 * 1000 # g N per L
mol_smz_dry_SB = gl/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
mgm3 = 31.4 + 12.7 # mg of small + med zoos per cubic meter 
gN = 20/1000 # convert mg to g of N per g of zooplankton biomass (NB); Fig. 5
gm3 = mgm3/1000 # convert to g biomass per cubic meter
gNm3 = gm3 * gN # g N contributed by small/med zoos per cubic meter
gL = gNm3 * 1000 # g N per L
mol_smz_wet_SB = gl/14.0067 # convert to mol/kg by dividing by molecular weight of N



mgg = 30 # mg of N per g of zooplankton biomass (CB); Fig. 5


mgg = 45 # mg of N per g of zooplankton biomass (SB); Fig. 5

### nbact
## Dry season
cmL = 1830000 # cells per mL; value from Table 3, Cox et al. 2006
cL = cmL * 1000 # convert to L
fgL = cL * 1.8 # multiply by femtograms of N per cell; value from Table 1, Cox et al. 2006
gL = fgL * 1e-15 # convert fg to g
mol = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
print("nbact-dry-NB:" ,mol)
## Wet season
cmL = 1440000 # cells per mL; value from Table 3, Cox et al. 2006
cL = cmL * 1000 # convert to L
fgL = cL * 1.8 # multiply by femtograms of N per cell; value from Table 1, Cox et al. 2006
gL = fgL * 1e-15 # convert fg to g
mol = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
print("nbact-wet-NB:" ,mol)
## Dry season
cmL = 2970000 # cells per mL; value from Table 3, Cox et al. 2006
cL = cmL * 1000 # convert to L
fgL = cL * 1.8 # multiply by femtograms of N per cell; value from Table 1, Cox et al. 2006
gL = fgL * 1e-15 # convert fg to g
mol = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
print("nbact-dry-CB:" ,mol)
## Wet season
cmL = 2200000 # cells per mL; value from Table 3, Cox et al. 2006
cL = cmL * 1000 # convert to L
fgL = cL * 1.8 # multiply by femtograms of N per cell; value from Table 1, Cox et al. 2006
gL = fgL * 1e-15 # convert fg to g
mol = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
print("nbact-wet-CB:" ,mol)
## Dry season
cmL = 3700000 # cells per mL; value from Table 3, Cox et al. 2006
cL = cmL * 1000 # convert to L
fgL = cL * 1.8 # multiply by femtograms of N per cell; value from Table 1, Cox et al. 2006
gL = fgL * 1e-15 # convert fg to g
mol = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
print("nbact-dry-NB:" ,mol)
## Wet season
cmL = 2800000 # cells per mL; value from Table 3, Cox et al. 2006
cL = cmL * 1000 # convert to L
fgL = cL * 1.8 # multiply by femtograms of N per cell; value from Table 1, Cox et al. 2006
gL = fgL * 1e-15 # convert fg to g
mol = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
print("nbact-wet-NB:" ,mol)

### nsm (nanoeukaryotes, picoeukaryotes, Syn. spp) (NB)

## Picoeukaryotes
## Dry season
r = 0.5 * 1 # assumed mean cell radius; value sourced from Ribeiro et al. 2016 
cmL = 14400 # cells per mL, Table 3
cL = cmL * 1000 # convert to cells per L
picoeukaryote_vol = ((4/3) * 3.14 * (r**3)) # calculate volume of picoeukaryote in cubic micrometers
picoeukaryote_fg_N = 26.1 * picoeukaryote_vol # fg N per cell
picoeukaryote_g_N = picoeukaryote_fg_N * 1e-15 # convert fg to g
picoeukaryote_gL = cL * picoeukaryote_g_N # calculate g per L
pico_dry_mol = picoeukaryote_gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
r = 0.5 * 1 # assumed mean cell radius; value sourced from Ribeiro et al. 2016 
cmL = 10200 # cells per mL, Table 3
cL = cmL * 1000 # convert L
picoeukaryote_vol = ((4/3) * 3.14 * (r**3)) # calculate volume of picoeukaryote in cubic micrometers
picoeukaryote_fg_N = 26.1 * picoeukaryote_vol # fg N per cell
picoeukaryote_g_N = picoeukaryote_fg_N * 1e-15 # convert fg to g
picoeukaryote_gL = cL * picoeukaryote_g_N # calculate g per L
pico_wet_mol = picoeukaryote_gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Nanoeukaryotes
## Dry season
r = 0.5 * 3.3 # mean cell radius; value sourced from Cox et al. 2006 
cmL = 366 # cells per mL, Table 3
cL = cmL * 1000 # convert to cells per L
nanoeukaryote_vol = ((4/3) * 3.14 * (r**3)) # calculate volume of nanoeukaryote in cubic micrometers
nanoeukaryote_fg_N = 26.1 * nanoeukaryote_vol # fg N per volume (= per cell)
nanoeukaryote_g_N = nanoeukaryote_fg_N * 1e-15 # convert fg to g N 
nanoeukaryote_gL = cL * nanoeukaryote_g_N # calculate g per L
nano_dry_mol = nanoeukaryote_gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
r = 0.5 * 3.3 # mean cell radius; value sourced from Cox et al. 2006 
cmL = 218 # cells per mL, Table 3
cL = cmL * 1000 # convert to cells per L
nanoeukaryote_vol = ((4/3) * 3.14 * (r**3)) # calculate volume of nanoeukaryote in cubic micrometers
nanoeukaryote_fg_N = 26.1 * nanoeukaryote_vol # fg N per volume (= per cell)
nanoeukaryote_g_N = nanoeukaryote_fg_N * 1e-15 # convert fg to g N 
nanoeukaryote_gL = cL * nanoeukaryote_g_N # calculate g per L
nano_wet_mol = nanoeukaryote_gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
### Synechococcus
## Dry season
cmL = 249000 # cells per mL
cL = cmL * 1000 # convert to L
fgL = cL * 21 # multiply by femtograms of N per cell; value from Table 1, Cox et al. 2006
gL = fgL * 1e-15 # convert fg to g
syn_dry_mol = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
cmL = 103000 # cells per mL
cL = cmL * 1000 # convert to L
fgL = cL * 21 # multiply by femtograms of N per cell; value from Table 1, Cox et al. 2006
gL = fgL * 1e-15 # convert fg to g
syn_wet_mol = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
# Sums
wet_sum = pico_wet_mol + nano_wet_mol + syn_wet_mol
print("Wet season nsm-NB:" ,wet_sum)
dry_sum = pico_dry_mol + nano_dry_mol + syn_dry_mol
print("Dry season nsm-NB:" ,dry_sum)

### nsm (nanoeukaryotes, picoeukaryotes, Syn. spp) (CB)

## Picoeukaryotes
## Dry season
r = 0.5 * 1 # assumed mean cell radius; value sourced from Ribeiro et al. 2016 
cmL = 22200 # cells per mL, Table 3
cL = cmL * 1000 # convert to cells per L
picoeukaryote_vol = ((4/3) * 3.14 * (r**3)) # calculate volume of picoeukaryote in cubic micrometers
picoeukaryote_fg_N = 26.1 * picoeukaryote_vol # fg N per cell
picoeukaryote_g_N = picoeukaryote_fg_N * 1e-15 # convert fg to g
picoeukaryote_gL = cL * picoeukaryote_g_N # calculate g per L
pico_dry_mol = picoeukaryote_gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
r = 0.5 * 1 # assumed mean cell radius; value sourced from Ribeiro et al. 2016 
cmL = 15900 # cells per mL, Table 3
cL = cmL * 1000 # convert L
picoeukaryote_vol = ((4/3) * 3.14 * (r**3)) # calculate volume of picoeukaryote in cubic micrometers
picoeukaryote_fg_N = 26.1 * picoeukaryote_vol # fg N per cell
picoeukaryote_g_N = picoeukaryote_fg_N * 1e-15 # convert fg to g
picoeukaryote_gL = cL * picoeukaryote_g_N # calculate g per L
pico_wet_mol = picoeukaryote_gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Nanoeukaryotes
## Dry season
r = 0.5 * 3.3 # mean cell radius; value sourced from Cox et al. 2006 
cmL = 216 # cells per mL, Table 3
cL = cmL * 1000 # convert to cells per L
nanoeukaryote_vol = ((4/3) * 3.14 * (r**3)) # calculate volume of nanoeukaryote in cubic micrometers
nanoeukaryote_fg_N = 26.1 * nanoeukaryote_vol # fg N per volume (= per cell)
nanoeukaryote_g_N = nanoeukaryote_fg_N * 1e-15 # convert fg to g N 
nanoeukaryote_gL = cL * nanoeukaryote_g_N # calculate g per L
nano_dry_mol = nanoeukaryote_gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
r = 0.5 * 3.3 # mean cell radius; value sourced from Cox et al. 2006 
cmL = 288 # cells per mL, Table 3
cL = cmL * 1000 # convert to cells per L
nanoeukaryote_vol = ((4/3) * 3.14 * (r**3)) # calculate volume of nanoeukaryote in cubic micrometers
nanoeukaryote_fg_N = 26.1 * nanoeukaryote_vol # fg N per volume (= per cell)
nanoeukaryote_g_N = nanoeukaryote_fg_N * 1e-15 # convert fg to g N 
nanoeukaryote_gL = cL * nanoeukaryote_g_N # calculate g per L
nano_wet_mol = nanoeukaryote_gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
### Synechococcus
## Dry season
cmL = 347000 # cells per mL
cL = cmL * 1000 # convert to L
fgL = cL * 21 # multiply by femtograms of N per cell; value from Table 1, Cox et al. 2006
gL = fgL * 1e-15 # convert fg to g
syn_dry_mol = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
cmL = 272000 # cells per mL
cL = cmL * 1000 # convert to L
fgL = cL * 21 # multiply by femtograms of N per cell; value from Table 1, Cox et al. 2006
gL = fgL * 1e-15 # convert fg to g
syn_wet_mol = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
# Sums
wet_sum = pico_wet_mol + nano_wet_mol + syn_wet_mol
print("Wet season nsm-CB:" ,wet_sum)
dry_sum = pico_dry_mol + nano_dry_mol + syn_dry_mol
print("Dry season nsm-CB:" ,dry_sum)

### nsm (nanoeukaryotes, picoeukaryotes, Syn. spp) (SB)

## Picoeukaryotes
## Dry season
r = 0.5 * 1 # assumed mean cell radius; value sourced from Ribeiro et al. 2016 
cmL = 35100 # cells per mL, Table 3
cL = cmL * 1000 # convert to cells per L
picoeukaryote_vol = ((4/3) * 3.14 * (r**3)) # calculate volume of picoeukaryote in cubic micrometers
picoeukaryote_fg_N = 26.1 * picoeukaryote_vol # fg N per cell
picoeukaryote_g_N = picoeukaryote_fg_N * 1e-15 # convert fg to g
picoeukaryote_gL = cL * picoeukaryote_g_N # calculate g per L
pico_dry_mol = picoeukaryote_gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
r = 0.5 * 1 # assumed mean cell radius; value sourced from Ribeiro et al. 2016 
cmL = 25300 # cells per mL, Table 3
cL = cmL * 1000 # convert L
picoeukaryote_vol = ((4/3) * 3.14 * (r**3)) # calculate volume of picoeukaryote in cubic micrometers
picoeukaryote_fg_N = 26.1 * picoeukaryote_vol # fg N per cell
picoeukaryote_g_N = picoeukaryote_fg_N * 1e-15 # convert fg to g
picoeukaryote_gL = cL * picoeukaryote_g_N # calculate g per L
pico_wet_mol = picoeukaryote_gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Nanoeukaryotes
## Dry season
r = 0.5 * 3.3 # mean cell radius; value sourced from Cox et al. 2006 
cmL = 339 # cells per mL, Table 3
cL = cmL * 1000 # convert to cells per L
nanoeukaryote_vol = ((4/3) * 3.14 * (r**3)) # calculate volume of nanoeukaryote in cubic micrometers
nanoeukaryote_fg_N = 26.1 * nanoeukaryote_vol # fg N per volume (= per cell)
nanoeukaryote_g_N = nanoeukaryote_fg_N * 1e-15 # convert fg to g N 
nanoeukaryote_gL = cL * nanoeukaryote_g_N # calculate g per L
nano_dry_mol = nanoeukaryote_gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
r = 0.5 * 3.3 # mean cell radius; value sourced from Cox et al. 2006 
cmL = 318 # cells per mL, Table 3
cL = cmL * 1000 # convert to cells per L
nanoeukaryote_vol = ((4/3) * 3.14 * (r**3)) # calculate volume of nanoeukaryote in cubic micrometers
nanoeukaryote_fg_N = 26.1 * nanoeukaryote_vol # fg N per volume (= per cell)
nanoeukaryote_g_N = nanoeukaryote_fg_N * 1e-15 # convert fg to g N 
nanoeukaryote_gL = cL * nanoeukaryote_g_N # calculate g per L
nano_wet_mol = nanoeukaryote_gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
### Synechococcus
## Dry season
cmL = 394000 # cells per mL
cL = cmL * 1000 # convert to L
fgL = cL * 21 # multiply by femtograms of N per cell; value from Table 1, Cox et al. 2006
gL = fgL * 1e-15 # convert fg to g
syn_dry_mol = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
cmL = 326000 # cells per mL
cL = cmL * 1000 # convert to L
fgL = cL * 21 # multiply by femtograms of N per cell; value from Table 1, Cox et al. 2006
gL = fgL * 1e-15 # convert fg to g
syn_wet_mol = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
# Sums
wet_sum = pico_wet_mol + nano_wet_mol + syn_wet_mol
print("Wet season nsm-SB:" ,wet_sum)
dry_sum = pico_dry_mol + nano_dry_mol + syn_dry_mol
print("Dry season nsm-SB:" ,dry_sum)

# nlg (diatoms, Chaetoceros, dinoflagellates) (NB)
## Diatoms
## Dry season
r = 0.5 * 54 # cell radius derived from Table 2, Cox et al. 2006
cL = 4920 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_diatoms_dry = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
r = 0.5 * 54 # cell radius derived from Table 2, Cox et al. 2006
cL = 3360 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_diatoms_wet = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
### Chaetoceros
## Dry season
r = 0.5 * 10 # cell radius derived from Table 2, Cox et al. 2006
cL = 137 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_ch_dry = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
r = 0.5 * 10 # cell radius derived from Table 2, Cox et al. 2006
cL = 812 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_ch_wet = gL/14.0067 # convert to mol/kg by dividing by molecular weight of 
### Dinoflagellates
## Dry season
r = 0.5 * 21 # cell radius in micrometers derived from Table 2, Cox et al. 2006
cL = 2950 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_dino_dry = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
r = 0.5 * 21 # cell radius derived from Table 2, Cox et al. 2006
cL = 2480 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_dino_wet = gL/14.0067 # convert to mol/kg by dividing by molecular weight of 
# Sums
wet_sum = mol_diatoms_wet + mol_ch_wet + mol_dino_wet
print("Wet season nlg-NB:" ,wet_sum)
dry_sum = mol_diatoms_dry + mol_ch_dry + mol_dino_dry
print("Dry season nlg-NB:" ,dry_sum)

# nlg (diatoms, Chaetoceros, dinoflagellates) (CB)
## Diatoms
## Dry season
r = 0.5 * 49 # cell radius derived from Table 2, Cox et al. 2006
cL = 1590 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_diatoms_dry = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
r = 0.5 * 49 # cell radius derived from Table 2, Cox et al. 2006
cL = 476 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_diatoms_wet = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
### Chaetoceros
## Dry season
r = 0.5 * 12 # cell radius derived from Table 2, Cox et al. 2006
cL = 0 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_ch_dry = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
r = 0.5 * 12 # cell radius derived from Table 2, Cox et al. 2006
cL = 210 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_ch_wet = gL/14.0067 # convert to mol/kg by dividing by molecular weight of 
### Dinoflagellates
## Dry season
r = 0.5 * 23 # cell radius in micrometers derived from Table 2, Cox et al. 2006
cL =  3290 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_dino_dry = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
r = 0.5 * 23 # cell radius derived from Table 2, Cox et al. 2006
cL = 1360 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_dino_wet = gL/14.0067 # convert to mol/kg by dividing by molecular weight of 
# Sums
wet_sum = mol_diatoms_wet + mol_ch_wet + mol_dino_wet
print("Wet season nlg-CB:" ,wet_sum)
dry_sum = mol_diatoms_dry + mol_ch_dry + mol_dino_dry
print("Dry season nlg-CB:" ,dry_sum)

# nlg (diatoms, Chaetoceros, dinoflagellates) (SB)
## Diatoms
## Dry season
r = 0.5 * 77 # cell radius derived from Table 2, Cox et al. 2006
cL = 1690 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_diatoms_dry = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
r = 0.5 * 77 # cell radius derived from Table 2, Cox et al. 2006
cL = 2330 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_diatoms_wet = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
### Chaetoceros
## Dry season
r = 0.5 * 11 # cell radius derived from Table 2, Cox et al. 2006
cL = 317 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_ch_dry = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
r = 0.5 * 11 # cell radius derived from Table 2, Cox et al. 2006
cL = 398 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_ch_wet = gL/14.0067 # convert to mol/kg by dividing by molecular weight of 
### Dinoflagellates
## Dry season
r = 0.5 * 22 # cell radius in micrometers derived from Table 2, Cox et al. 2006
cL = 4190 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_dino_dry = gL/14.0067 # convert to mol/kg by dividing by molecular weight of N
## Wet season
r = 0.5 * 22 # cell radius derived from Table 2, Cox et al. 2006
cL = 1990 # cells per L, Table 3
pgNc = 0.0172 * (((4/3) * 3.14 * (r**3))**1.023) # calculate pg N/cell per Table 1
pgNL = cL * pgNc # pg N/L
gL = pgNL * 1e-12 # convert pg to g
mol_dino_wet = gL/14.0067 # convert to mol/kg by dividing by molecular weight of 
# Sums
wet_sum = mol_diatoms_wet + mol_ch_wet + mol_dino_wet
print("Wet season nlg-SB:" ,wet_sum)
dry_sum = mol_diatoms_dry + mol_ch_dry + mol_dino_dry
print("Dry season nlg-SB:" ,dry_sum)

Cil-dry-NB: 2.657229673508002e-05
Cil-wet-NB: 3.44735289082004e-05
Cil-dry-CB: 1.9979548870884028e-05
Cil-wet-CB: 2.214358061256668e-05
Cil-dry-SB: 2.2948801725750923e-05
Cil-wet-SB: 3.638592905201298e-05
nbact-dry-NB: 2.3517316712716056e-07
nbact-wet-NB: 1.850542954443231e-07
nbact-dry-CB: 3.816744843539163e-07
nbact-wet-CB: 2.827218402621603e-07
nbact-dry-NB: 4.754867313499968e-07
nbact-wet-NB: 3.5982779669729485e-07
Wet season nsm-NB: 1.7201268245154107e-07
Dry season nsm-NB: 4.0019031140654114e-07
Wet season nsm-CB: 4.3340307018098486e-07
Dry season nsm-CB: 5.494696332703634e-07
Wet season nsm-SB: 5.245823275713765e-07
Dry season nsm-SB: 6.368262060825891e-07
Wet season nlg-NB: 4.5966552709096675e-07
Dry season nlg-NB: 6.673712732327241e-07
Wet season nlg-CB: 5.9658978058149e-08
Dry season nlg-CB: 1.8638754399719018e-07
Wet season nlg-SB: 9.258651703520884e-07
Dry season nlg-SB: 6.944974023807759e-07


In [17]:
print(46/67)
print(.00000006/.00000019)
print(93/70)


0.6865671641791045
0.3157894736842105
1.3285714285714285


##### More parameterization notes that didn't fit elsewhere

  * "Due to high flushing rates, recovery times of bay waters from storm perturbations ranged from 3 to 8 d and appeared to be correlated with tidal range. Storm inputs have a significant effect on the water column ecosystem and biogeochemistry in southern Kaneohe Bay. The perturbations were only transient events and the system rapidly recovered to prestorm conditions." [Ringuet & Mackenzie 2005](https://www.researchgate.net/profile/F_Mackenzie/publication/227303537_Controls_on_Nutrient_and_Phytoplankton_Dynamics_during_Normal_Flow_and_Storm_Runoff_Conditions_Southern_Kaneohe_Bay_Hawaii/links/00b7d5293aa8ee5d17000000.pdf)
  * [Ringuet & Mackenzie 2005](https://www.researchgate.net/profile/F_Mackenzie/publication/227303537_Controls_on_Nutrient_and_Phytoplankton_Dynamics_during_Normal_Flow_and_Storm_Runoff_Conditions_Southern_Kaneohe_Bay_Hawaii/links/00b7d5293aa8ee5d17000000.pdf) have more information about seasonal changes in nutrient cycles across the bay.
  * [Ringuet & Mackenzie 2005](https://www.researchgate.net/profile/F_Mackenzie/publication/227303537_Controls_on_Nutrient_and_Phytoplankton_Dynamics_during_Normal_Flow_and_Storm_Runoff_Conditions_Southern_Kaneohe_Bay_Hawaii/links/00b7d5293aa8ee5d17000000.pdf) note that DIN:DIP across the bay = 3-5
  * [Drupp et al. 2011](https://www.researchgate.net/profile/F_Mackenzie/publication/253153306_Nutrient_inputs_dynamics_and_phytoplankton_response_in_a_semi-enclosed_subtropical_embayment_Kaneohe_Bay_Hawaii/links/566372d808ae418a786bb1ce.pdf) contains good information on seasonal chemical changes across the bay.
  * "The concentration of ammonium in streams was assumed to be zero in KECOM because dissolved inorganic nitrogen is predominantly nitrate in Hawaiian streams (Smith et al., 1981). However, the ammonium concentration is not altogether negligible, as suggested by the observations of Hoover (2002), but is not significant enough to include in the model calculations." [Tanaka & Mackenzie 2005](https://www.researchgate.net/profile/F_Mackenzie/publication/265088005_Ecosystem_behavior_of_southern_Kaneohe_Bay_A_statistical_and_modeling_approach/links/566f30a708aec0bb67bf0739.pdf)
    * Note that ammonia is represented as nh4 in the model; technically, its chemical formula is nh3. I am *assuming* based on the lack of any mentions of ammonium whatsoever that nh4, specified as ammonia, represents ammonia.
  *  [Cox et al. 2006](https://www.int-res.com/articles/meps2006/324/m324p019.pdf): "Pulses were characterized by an elevation in inorganic nutrient concentrations in the Bay close to the stream mouth. The general response was an increase in abundance of microphytoplankton and chl a after a 3 to 6 d lag following the nutrient increase. Picophytoplankton showed an increase in fluorescence per cell after a 12 to 24 h lag, probably related to a decrease in irradiance associated with turbidity in runoff. The Bay can act as source of dissolved inorganic nutrients and plankton for oceanic waters; however, planktonic populations in the Bay are primarily autochthonous and do not represent an oceanic source of nutrients for plankton consumers within Kāne‘ohe Bay."